<a href="https://colab.research.google.com/github/PeterS111/GPT-2_DEMO_FINE_TUNING/blob/main/WORX_GPT_2_DEMO_FINE_TUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# GPT-2 Base Fine-Tuning Demo
version 10.12.2020

The code is based on https://github.com/priya-dwivedi/Deep-Learning/ by N Shepperd, with small changes by Peter S

This notebook allows to fine-tune GPT-2 Small and GPT-2 Medium and generate text from them. 

First make sure that you are using a GPU: Runtime/Change runtime type/ -> Select "GPU". There are four types of GPUs on Colab: K80, P4, T4 and P100. If you want to fine-tune GPT-2 Medium you will need T4 or P100. For the GPT-2 Small any GPU will work. To check which GPU you have, run the command below. If the GPU is not the one you require go to "Runtime/Factory reset runtime" and run the command below again. Usually after a few "resets" you will get either T4 or P100.

In [ ]:
!nvidia-smi

# 2. Download the repository from github:

In [ ]:
!git clone https://github.com/PeterS111/GPT-2_DEMO_FINE_TUNING  

# 3. Change the working directory to the main folder:

In [ ]:
cd /content/GPT-2_DEMO_FINE_TUNING

# 4. Install Tranformers library

In [ ]:
pip install "transformers==2.7.0"

# 5. Training the model without validation dataset:

You can train the model with or without the validation dataset.
Datasets of Tolstoy's novels and Shelley's are provided with the notebook. You can of course replace those datasets with your own. In that case you will have to upload them to /content/GPT-2_DEMO_FINE_TUNING/input_data. You can "drag and drop" them from your PC. You will have to change the argument: "--train_data_file" accordingly.

To control the training time you can change the number of epochs.

To train the Medium model change the following argument:

--model_name_or_path=gpt2

to:

--model_name_or_path=gpt2-medium


In [ ]:
!python run_lm_finetuning.py --output_dir=output  --model_type=gpt2 --model_name_or_path=gpt2 --do_train  --train_data_file=input_data/Shelley.txt --overwrite_output_dir --block_size=200 --per_gpu_train_batch_size=1 --save_steps 1000 --save_total_limit 1 --num_train_epochs=5 --logging_steps=5

# 6. Training the model with validation dataset:

The same as above but if you want to used your own datasets you will have to change the "--eval_data_file" and "--train_data_file" arguments accordingly.

In [ ]:
!python run_lm_finetuning.py --output_dir=output  --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=input_data/Shelley_train.txt --do_eval --eval_data_file=input_data/Shelley_val.txt --overwrite_output_dir --block_size=200 --per_gpu_train_batch_size=1 --save_steps 1000 --save_total_limit 1 --num_train_epochs=5 --logging_steps=5

# 7. Generate text:

First check the folder /GPT-2_DEMO_FINE_TUNING/output for a folder named "checkpoint- *". In our case the folder is "checkpoint-6000". If you used a different dataset or change the training time, the folder may be different. Make sure that the parameter "--model_name_or_path output" has the correct argument. You can change the "--prompt" parameter to any continuous string. If you want to generate more samples you can change the "--num_samples" parameter.

Generated text will appear in the output cell and a copy will be saved in the "GPT-2_DEMO_FINE_TUNING/outputs" folder.





In [ ]:
!python run_generation.py --model_type gpt2 --temperature 1.0 --top_k 50 --top_p 1.0 --model_name_or_path output/checkpoint-6000 --length 500 --prompt "The blue sky" --seed 37 --num_samples 1


# 8. Exporting the fine-tuned model to Google Drive

If you want to save the fine-tuned model for later use, follow these steps:

8.1. Zip the model.

In [ ]:
!tar -czvf "my_model_6000_steps.tar.gz" output/checkpoint-6000/*

8.2. Mount Google Drive (it will require authentication)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

8.3. Export the compressed model to Drive. This only takes few minutes, but sometimes you may have to run the command twice.

In [ ]:
!cp "my_model_6000_steps.tar.gz" "/content/drive/My Drive/"

#9. Importing saved model from Google Drive

If you want to return to saved model, do the following:

First, run points 2, 3 and 4 (download the repository, change the directory, install Transformers), the same as above. Then:

9.1. Create the output folder:

In [ ]:
!mkdir output/ 

 9.2. Mount Google Drive, the same as 8.2:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

9.3. Import the previously saved model:

In [ ]:
!cp "/content/drive/My Drive/my_model_6000_steps.tar.gz" /content/GPT-2_DEMO_FINE_TUNING/output

9.4. Unpack the model and remove the .tar file:

In [ ]:
!tar xf /content/GPT-2_DEMO_FINE_TUNING/output/my_model_6000_steps.tar.gz
!rm -v /content/GPT-2_DEMO_FINE_TUNING/output/my_model_6000_steps.tar.gz

9.5. Ready to generate text. The command below is the same as in point 7, check there for details.

In [ ]:
!python run_generation.py --model_type gpt2 --temperature 1.0 --top_k 50 --top_p 1.0 --model_name_or_path output/checkpoint-6000 --length 500 --prompt "The blue sky" --seed 37 --num_samples 1